In [1]:
import numpy as np
from Bio import SeqIO

# Define valid amino acids
valid_amino_acids = "ARNDCQEGHILKMFPSTWYV"

def preprocess_fasta(fasta_file):
    """
    Reads a FASTA file and filters out non-standard amino acids.
    """
    sequence = ""

    # Read FASTA file properly
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence += str(record.seq)

    # Keep only valid amino acids
    filtered_sequence = "".join([aa for aa in sequence if aa in valid_amino_acids])

    if len(filtered_sequence) == 0:
        raise ValueError("❌ Error: No valid amino acids found in input sequence!")

    return filtered_sequence

# Example usage
fasta_file = r"data\rcsb_pdb_4JRB.fasta"
raw_sequence = preprocess_fasta(fasta_file)
print("Cleaned Sequence:", raw_sequence)


Cleaned Sequence: GSHMSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFGYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGIAAARNLQDDLQDFLALIPVDQIIAIATDYLANDAEVQAAVAYLQSDEFETIVVALDALPELQNFLNFLEANGLNAIDFLNGIHDLLGIPHIPVSGRKYHIRRGVGITGLIDDVLAILPIEDLKALFNEKLETSPDFLALYNAIRSPEFQSIVQTLNAMPEYQNLLQKLREKGVDVDKIIELIRALF


In [2]:
hydrophobic_residues = {'A', 'V', 'I', 'L', 'M', 'F', 'W'}
polar_residues = {'S', 'T', 'Y', 'N', 'Q'}
charged_residues = {'R', 'K', 'D', 'E'}

In [3]:
from sklearn.svm import SVC
import numpy as np

def extract_features(sequence):
    """
    Converts amino acid sequence into a feature vector.
    """
    features = []
    for aa in sequence:
        features.append([
            1 if aa in hydrophobic_residues else 0,
            1 if aa in polar_residues else 0,
            1 if aa in charged_residues else 0
        ])
    return np.array(features)

# Train an SVM to predict secondary structure tendencies
X_train = extract_features(raw_sequence)  # Convert sequence to feature vectors
y_train = np.random.randint(0, 2, size=len(X_train))  # Placeholder labels

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

print("✅ SVM Feature Extraction Complete!")


✅ SVM Feature Extraction Complete!


In [4]:
import numpy as np

def initialize_positions(sequence, min_distance=1.0, scale=5.0, max_attempts=5000):
    """
    Initializes 3D positions for residues ensuring minimum distance constraints.
    Tries multiple times before failing.
    """
    positions = []
    
    for aa in sequence:
        for _ in range(max_attempts):  # Try multiple times before failing
            new_position = np.random.uniform(-scale, scale, 3)  # Spread positions out more
            
            # Check minimum distance constraint
            if all(np.linalg.norm(new_position - np.array(p)) >= min_distance for p in positions):
                positions.append(new_position)
                break
        else:
            print(f"❌ Failed to place residue '{aa}' after {max_attempts} attempts.")
            return np.array(positions)  # Return partial positions instead of failing completely

    return np.array(positions)

positions = initialize_positions(raw_sequence, min_distance=1.0, scale=5.0)

if positions.shape[0] == len(raw_sequence):
    print("✅ Successfully placed all residues!")
else:
    print("⚠️ Some residues were not placed. Try adjusting `min_distance` or `scale`.")


✅ Successfully placed all residues!


In [1]:

import torch
import esm

model = esm.pretrained.esmfold_v1()
model = model.eval().cuda()

# Optionally, uncomment to set a chunk size for axial attention. This can help reduce memory.
# Lower sizes will have lower memory requirements at the cost of increased speed.
# model.set_chunk_size(128)

sequence = "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"
# Multimer prediction can be done with chains separated by ':'

with torch.no_grad():
    output = model.infer_pdb(sequence)

with open("result.pdb", "w") as f:
    f.write(output)

import biotite.structure.io as bsio
struct = bsio.load_structure("result.pdb", extra_fields=["b_factor"])
print(struct.b_factor.mean())  # this will be the pLDDT
# 88.3

ModuleNotFoundError: No module named 'omegaconf'

In [112]:
import MDAnalysis as mda

# Load AI-generated structure
u = mda.Universe("ai_generated_structure.pdb")

# Select atoms
ca_atoms = u.select_atoms("name CA")

# Initialize positions
positions = ca_atoms.positions.copy()





In [116]:
import numpy as np

def enforce_peptide_constraints(positions, sequence):
    """
    Ensures proper backbone angles and prevents unnatural peptide bond distortions.
    """
    epsilon = 1e-6  # Avoid division by zero
    constraints = np.zeros_like(positions)

    for i in range(len(positions) - 1):
        diff = positions[i+1] - positions[i]
        distance = np.linalg.norm(diff) + epsilon
        
        # Enforce peptide bond length (~1.32 Å for CA-CA distance)
        ideal_length = 1.32
        correction_factor = (distance - ideal_length) / distance
        constraints[i] += diff * correction_factor

    return positions - constraints  # Adjust positions to enforce constraints

# Apply constraints to the AI-predicted structure
positions = enforce_peptide_constraints(positions, raw_sequence)


In [117]:
def detect_hydrogen_bonds(positions, sequence):
    """
    Identifies hydrogen bonds between amino acids.
    """
    hydrogen_bonds = []
    for i, _ in enumerate(sequence):
        for j in range(i + 4, len(sequence)):  # Minimum spacing for alpha-helix
            distance = np.linalg.norm(positions[i] - positions[j])
            if 2.5 < distance < 3.5:  # Hydrogen bond distance range
                hydrogen_bonds.append((i, j))
    
    print(f"Detected {len(hydrogen_bonds)} hydrogen bonds.")
    return hydrogen_bonds

# Detect hydrogen bonds in AI-predicted structure
hydrogen_bonds = detect_hydrogen_bonds(positions, raw_sequence)


Detected 4 hydrogen bonds.


In [118]:
def cluster_hydrophobic_core(positions, sequence):
    """
    Adjusts positions of hydrophobic residues to drive core formation.
    """
    for i, residue in enumerate(sequence):
        if residue in hydrophobic_residues:
            positions[i] -= 1.0  # Move inward
    return positions

# Apply hydrophobic clustering
positions = cluster_hydrophobic_core(positions, raw_sequence)


In [119]:
# def enforce_quaternary_interactions(positions, chains):
#     """
#     Adjusts inter-chain distances to ensure proper quaternary structure formation.
#     """
#     for chain_a, chain_b in chains:
#         for i in chain_a:
#             for j in chain_b:
#                 distance = np.linalg.norm(positions[i] - positions[j])
#                 if distance < 3.0:  # Prevent steric clashes
#                     positions[i] += 0.5  # Push apart slightly
#     return positions

# # Example: Two chains A & B
# chains = [(range(0, 50), range(50, 100))]  # Example chain indices
# positions = enforce_quaternary_interactions(positions, chains)


In [120]:
helix_formers = {'A', 'L', 'M', 'E', 'Q', 'K', 'R'}
sheet_formers = {'V', 'I', 'Y', 'F', 'C', 'T', 'W'}
helix_breakers = {'P', 'G'}


In [121]:
def calculate_energy(positions, sequence):
    """
    Physics-based scoring function for structure optimization.
    """
    epsilon = 1e-6  
    energy = 0

    for i, residue1 in enumerate(sequence):
        for j, residue2 in enumerate(sequence):
            if i < j:
                distance = np.linalg.norm(positions[i] - positions[j]) + epsilon
                
                # Hydrophobic interactions
                if residue1 in helix_formers and residue2 in helix_formers:
                    energy -= 1 / distance  # Favor close helix-forming residues

                # Electrostatic interactions
                if (residue1 in {'R', 'K'} and residue2 in {'D', 'E'}) or \
                   (residue1 in {'D', 'E'} and residue2 in {'R', 'K'}):
                    energy -= 2 / distance  # Opposite charge attraction

                # Steric clashes
                if distance < 1.5:
                    energy += 10 / (distance**2)  # Penalize steric clashes

    return energy


In [123]:
import numpy as np

# Define amino acid property groups
hydrophobic_residues = {'A', 'V', 'I', 'L', 'M', 'F', 'W', 'C', 'Y'}
charged_residues = {'R', 'K', 'D', 'E'}
polar_residues = {'S', 'T', 'N', 'Q', 'Y'}
bulky_residues = {'F', 'Y', 'W', 'R', 'K'}
hydrogen_bond_donors = {'S', 'T', 'N', 'Q', 'Y'}

def compute_force_scales(sequence):
    """
    Computes the scaling factors for hydrophobic, electrostatic, repulsion, 
    and hydrogen bonding forces based on sequence composition.
    """
    total_length = len(sequence)
    
    if total_length == 0:
        raise ValueError("Empty sequence provided!")

    # Calculate fractions
    hydrophobic_fraction = sum(1 for aa in sequence if aa in hydrophobic_residues) / total_length
    charged_fraction = sum(1 for aa in sequence if aa in charged_residues) / total_length
    bulky_fraction = sum(1 for aa in sequence if aa in bulky_residues) / total_length
    hydrogen_bond_fraction = sum(1 for aa in sequence if aa in hydrogen_bond_donors) / total_length

    # Normalize values between 0.1 and 1.0
    hydrophobic_force_scale = 0.1 + 0.9 * hydrophobic_fraction
    electrostatic_force_scale = 0.1 + 0.9 * charged_fraction
    repulsion_scale = 0.1 + 0.9 * bulky_fraction
    hydrogen_bond_scale = 0.1 + 0.9 * hydrogen_bond_fraction

    return hydrophobic_force_scale, electrostatic_force_scale, repulsion_scale, hydrogen_bond_scale

# Compute force scaling factors
hydrophobic_scale, electrostatic_scale, repulsion_scale, hydrogen_bond_scale = compute_force_scales(raw_sequence)

# Print results
print(f"Hydrophobic Force Scale: {hydrophobic_scale:.3f}")
print(f"Electrostatic Force Scale: {electrostatic_scale:.3f}")
print(f"Repulsion Force Scale: {repulsion_scale:.3f}")
print(f"Hydrogen Bonding Scale: {hydrogen_bond_scale:.3f}")


Hydrophobic Force Scale: 0.492
Electrostatic Force Scale: 0.311
Repulsion Force Scale: 0.264
Hydrogen Bonding Scale: 0.296


In [124]:
def optimize_positions(positions, sequence, iterations=1000, learning_rate=0.01):
    """
    Monte Carlo-based energy minimization to refine AI-generated structure.
    """
    epsilon = 1e-6  
  
    for iter in range(iterations):
        gradients = np.zeros_like(positions)

        for i, residue in enumerate(sequence):
            for j in range(len(sequence)):
                if i != j:
                    diff = positions[i] - positions[j]
                    distance = np.linalg.norm(diff) + epsilon

                    # Hydrophobic interactions
                    if residue in helix_formers and sequence[j] in helix_formers:
                        gradients[i] -= hydrophobic_scale * (diff / distance**2)

                    # Electrostatic interactions
                    if (residue in {'R', 'K'} and sequence[j] in {'D', 'E'}) or \
                       (residue in {'D', 'E'} and sequence[j] in {'R', 'K'}):
                        gradients[i] -= electrostatic_scale * (diff / distance**2)

                    # Steric clash repulsion
                    if distance < 1.5:
                        gradients[i] += repulsion_scale * (diff / distance**3)

        # Update positions
        positions -= learning_rate * gradients
        learning_rate *= 0.99  # Decay learning rate

        # Monitor energy every 100 iterations
        if iter % 100 == 0:
            energy = calculate_energy(positions, sequence)
            print(f"Iteration {iter}: Energy = {energy:.4f}")

    return positions

optimize_positions = optimize_positions(positions, raw_sequence)

Iteration 0: Energy = -280.0836
Iteration 100: Energy = -277.6806
Iteration 200: Energy = -276.8194
Iteration 300: Energy = -276.5065
Iteration 400: Energy = -276.3923
Iteration 500: Energy = -276.3506
Iteration 600: Energy = -276.3353
Iteration 700: Energy = -276.3297
Iteration 800: Energy = -276.3276
Iteration 900: Energy = -276.3269


In [101]:
import torch
import numpy as np

def save_pdb(coords, sequence, output_file="predicted_structure.pdb"):
    """
    Saves predicted coordinates as a PDB file with the correct residue names.

    Args:
        coords (np.ndarray): Predicted 3D coordinates of shape (seq_length, 3).
        sequence (str): Raw amino acid sequence (single-letter code).
        output_file (str): File path to save the PDB file.
    """
    # Mapping of single-letter amino acids to three-letter codes
    aa_to_three_letter = {
        "A": "ALA", "C": "CYS", "D": "ASP", "E": "GLU", "F": "PHE",
        "G": "GLY", "H": "HIS", "I": "ILE", "K": "LYS", "L": "LEU",
        "M": "MET", "N": "ASN", "P": "PRO", "Q": "GLN", "R": "ARG",
        "S": "SER", "T": "THR", "V": "VAL", "W": "TRP", "Y": "TYR"
    }

    # Ensure coordinates are a NumPy array
    if isinstance(coords, torch.Tensor):
        coords = coords.detach().cpu().numpy()

    # Save PDB
    with open(output_file, "w") as f:
        for i, (x, y, z) in enumerate(coords, start=1):
            aa = aa_to_three_letter.get(sequence[i - 1], "UNK")  # Get three-letter code
            f.write(
                f"ATOM  {i:5d}  CA  {aa} A{i:4d}    {x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           C\n"
            )
        f.write("TER\nEND\n")

# Example Usage
predicted_coords = np.random.rand(len(raw_sequence), 3) * 10  # Dummy structure for testing
save_pdb(predicted_coords, raw_sequence, "predicted_structure.pdb")
print("Predicted structure saved to predicted_structure.pdb")


Predicted structure saved to predicted_structure.pdb


In [1]:

import MDAnalysis as mda
from MDAnalysis.analysis.rms import rmsd

def calculate_rmsd(reference_pdb, predicted_pdb):
    """
    Computes RMSD between AI-generated structure and experimental PDB reference.
    """
    u_ref = mda.Universe(reference_pdb)
    u_pred = mda.Universe(predicted_pdb)

    ref_atoms = u_ref.select_atoms("name CA")
    pred_atoms = u_pred.select_atoms("name CA")

    rmsd_value = rmsd(pred_atoms.positions, ref_atoms.positions, superposition=True)
    print(f"RMSD After Refinement: {rmsd_value:.3f} Å")
    return rmsd_value

# Compare AI structure vs. experimental reference
calculate_rmsd("data/3zqo.pdb", "refined_structure.pdb")


C:\Users\sravy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\MDAnalysis\topology\PDBParser.py:350: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn("Element information is missing, elements attribute "
C:\Users\sravy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\MDAnalysis\coordinates\PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


ValueError: a and b must have same shape